# Ejercicio 2: Desarrollo Web y Integración con APIs

## Objetivos
- Aplicar conceptos de funciones y manejo de archivos en desarrollo web
- Introducir programación orientada a objetos en contexto web
- Complementar el conocimiento base con tecnologías modernas

## Contexto
Este ejercicio extiende los conceptos del repositorio hacia aplicaciones web modernas, APIs y servicios externos.

## Parte 1: Configuración y Herramientas Web

In [ ]:
# Instalación de dependencias (ejecutar solo si es necesario)
# !pip install requests flask beautifulsoup4 streamlit

import requests
import json
from datetime import datetime
import os
from urllib.parse import urljoin
import time

# Simulador de API local para ejercicios
class SimuladorAPI:
    """
    Clase que simula una API REST para ejercicios
    Aplica conceptos de POO del repositorio
    """
    
    def __init__(self):
        self.datos = {
            'usuarios': [
                {'id': 1, 'nombre': 'Juan', 'email': 'juan@example.com', 'activo': True},
                {'id': 2, 'nombre': 'María', 'email': 'maria@example.com', 'activo': True},
                {'id': 3, 'nombre': 'Carlos', 'email': 'carlos@example.com', 'activo': False}
            ],
            'productos': [
                {'id': 1, 'nombre': 'Laptop', 'precio': 999.99, 'stock': 10},
                {'id': 2, 'nombre': 'Mouse', 'precio': 29.99, 'stock': 50},
                {'id': 3, 'nombre': 'Teclado', 'precio': 79.99, 'stock': 25}
            ]
        }
    
    def obtener_usuarios(self, activos_solo=True):
        """Simula GET /usuarios"""
        if activos_solo:
            return [u for u in self.datos['usuarios'] if u['activo']]
        return self.datos['usuarios']
    
    def obtener_usuario(self, user_id):
        """Simula GET /usuarios/{id}"""
        for usuario in self.datos['usuarios']:
            if usuario['id'] == user_id:
                return usuario
        return None
    
    def crear_usuario(self, datos_usuario):
        """Simula POST /usuarios"""
        nuevo_id = max([u['id'] for u in self.datos['usuarios']]) + 1
        nuevo_usuario = {
            'id': nuevo_id,
            'nombre': datos_usuario['nombre'],
            'email': datos_usuario['email'],
            'activo': datos_usuario.get('activo', True)
        }
        self.datos['usuarios'].append(nuevo_usuario)
        return nuevo_usuario

# Crear instancia del simulador
api = SimuladorAPI()
print("Simulador de API inicializado")
print(f"Usuarios disponibles: {len(api.obtener_usuarios())}")

## Parte 2: Cliente API con Manejo de Errores

In [ ]:
# Ejercicio 2.1: Cliente API con funciones (concepto del repositorio)
class ClienteAPI:
    """
    Cliente para interactuar con APIs externas
    Demuestra POO y manejo de excepciones
    """
    
    def __init__(self, base_url="https://jsonplaceholder.typicode.com"):
        self.base_url = base_url
        self.sesion = requests.Session()
        self.sesion.headers.update({
            'User-Agent': 'Python-API-Client/1.0',
            'Accept': 'application/json'
        })
    
    def realizar_peticion(self, endpoint, method='GET', datos=None):
        """
        Función genérica para realizar peticiones HTTP
        Aplica conceptos de funciones y manejo de errores
        """
        url = urljoin(self.base_url, endpoint)
        
        try:
            if method == 'GET':
                response = self.sesion.get(url)
            elif method == 'POST':
                response = self.sesion.post(url, json=datos)
            elif method == 'PUT':
                response = self.sesion.put(url, json=datos)
            elif method == 'DELETE':
                response = self.sesion.delete(url)
            
            response.raise_for_status()
            return response.json() if response.content else {}
            
        except requests.exceptions.RequestException as e:
            print(f"Error en petición HTTP: {e}")
            return None
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
            return None
    
    def obtener_posts(self, limite=5):
        """Obtiene posts de ejemplo"""
        posts = self.realizar_peticion('/posts')
        return posts[:limite] if posts else []
    
    def obtener_usuarios_externos(self, limite=3):
        """Obtiene usuarios de la API externa"""
        usuarios = self.realizar_peticion('/users')
        return usuarios[:limite] if usuarios else []

# Probar el cliente API
cliente = ClienteAPI()
print("=== Probando API Externa ===")

# Obtener posts
posts = cliente.obtener_posts(3)
if posts:
    print(f"\nPosts obtenidos: {len(posts)}")
    for post in posts:
        print(f"- {post['title'][:50]}...")

# Obtener usuarios
usuarios_externos = cliente.obtener_usuarios_externos(2)
if usuarios_externos:
    print(f"\nUsuarios externos: {len(usuarios_externos)}")
    for usuario in usuarios_externos:
        print(f"- {usuario['name']} ({usuario['email']})")

## Parte 3: Procesamiento de Datos Web

In [ ]:
# Ejercicio 3.1: Procesamiento con funciones lambda y map/filter
def procesar_datos_web(datos):
    """
    Procesa datos obtenidos de APIs web
    Aplica conceptos de funciones de orden superior del repositorio
    """
    
    # Usar lambda para limpiar títulos
    limpiar_titulo = lambda texto: texto.strip().title()
    
    # Usar map para transformar datos
    datos_procesados = list(map(lambda item: {
        'id': item['id'],
        'titulo': limpiar_titulo(item['title']),
        'longitud': len(item['title']),
        'palabras': len(item['title'].split()),
        'usuario_id': item['userId']
    }, datos))
    
    # Usar filter para obtener títulos largos
    titulos_largos = list(filter(lambda x: x['longitud'] > 30, datos_procesados))
    
    # Estadísticas usando reduce (conceptualmente)
    longitud_promedio = sum(item['longitud'] for item in datos_procesados) / len(datos_procesados)
    
    return {
        'procesados': datos_procesados,
        'titulos_largos': titulos_largos,
        'estadisticas': {
            'total': len(datos_procesados),
            'longitud_promedio': longitud_promedio,
            'titulos_largos_count': len(titulos_largos)
        }
    }

# Procesar los posts obtenidos
if posts:
    resultado = procesar_datos_web(posts)
    
    print("\n=== Datos Procesados ===")
    print(f"Total de posts: {resultado['estadisticas']['total']}")
    print(f"Longitud promedio de títulos: {resultado['estadisticas']['longitud_promedio']:.1f}")
    print(f"Títulos largos (>30 chars): {resultado['estadisticas']['titulos_largos_count']}")
    
    print("\nTítulos procesados:")
    for item in resultado['procesados'][:3]:
        print(f"- {item['titulo']} ({item['longitud']} chars, {item['palabras']} palabras)")

## Parte 4: Almacenamiento y Persistencia

In [ ]:
# Ejercicio 4.1: Manejo de archivos con datos web (concepto del repositorio)
import json
import csv
from datetime import datetime

class GestorDatos:
    """
    Clase para gestionar almacenamiento de datos web
    Aplica conceptos de manejo de archivos del repositorio
    """
    
    def __init__(self, directorio_datos='datos_web'):
        self.directorio = directorio_datos
        self.crear_directorio()
    
    def crear_directorio(self):
        """Crea el directorio de datos si no existe"""
        if not os.path.exists(self.directorio):
            os.makedirs(self.directorio)
    
    def guardar_json(self, datos, nombre_archivo):
        """Guarda datos en formato JSON"""
        ruta_archivo = os.path.join(self.directorio, f"{nombre_archivo}.json")
        
        try:
            with open(ruta_archivo, 'w', encoding='utf-8') as archivo:
                json.dump(datos, archivo, indent=2, ensure_ascii=False)
            print(f"Datos guardados en: {ruta_archivo}")
            return True
        except Exception as e:
            print(f"Error al guardar JSON: {e}")
            return False
    
    def cargar_json(self, nombre_archivo):
        """Carga datos desde archivo JSON"""
        ruta_archivo = os.path.join(self.directorio, f"{nombre_archivo}.json")
        
        try:
            with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
                datos = json.load(archivo)
            print(f"Datos cargados desde: {ruta_archivo}")
            return datos
        except FileNotFoundError:
            print(f"Archivo no encontrado: {ruta_archivo}")
            return None
        except Exception as e:
            print(f"Error al cargar JSON: {e}")
            return None
    
    def exportar_csv(self, datos, nombre_archivo, campos):
        """Exporta datos a formato CSV"""
        ruta_archivo = os.path.join(self.directorio, f"{nombre_archivo}.csv")
        
        try:
            with open(ruta_archivo, 'w', newline='', encoding='utf-8') as archivo:
                escritor = csv.DictWriter(archivo, fieldnames=campos)
                escritor.writeheader()
                escritor.writerows(datos)
            print(f"CSV exportado a: {ruta_archivo}")
            return True
        except Exception as e:
            print(f"Error al exportar CSV: {e}")
            return False
    
    def crear_reporte(self, datos_posts, datos_usuarios):
        """Crea un reporte completo con timestamp"""
        reporte = {
            'timestamp': datetime.now().isoformat(),
            'posts': datos_posts,
            'usuarios': datos_usuarios,
            'resumen': {
                'total_posts': len(datos_posts) if datos_posts else 0,
                'total_usuarios': len(datos_usuarios) if datos_usuarios else 0
            }
        }
        return reporte

# Usar el gestor de datos
gestor = GestorDatos()

# Guardar datos obtenidos
if posts:
    gestor.guardar_json(posts, 'posts_api')
    
if usuarios_externos:
    gestor.guardar_json(usuarios_externos, 'usuarios_api')

# Crear y guardar reporte
reporte = gestor.crear_reporte(posts, usuarios_externos)
gestor.guardar_json(reporte, f'reporte_{datetime.now().strftime("%Y%m%d_%H%M%S")}')

# Exportar a CSV si tenemos datos procesados
if 'resultado' in locals() and resultado['procesados']:
    campos_csv = ['id', 'titulo', 'longitud', 'palabras', 'usuario_id']
    gestor.exportar_csv(resultado['procesados'], 'posts_procesados', campos_csv)

print("\n=== Archivos Creados ===")
if os.path.exists('datos_web'):
    archivos = os.listdir('datos_web')
    for archivo in archivos:
        print(f"- {archivo}")

## Parte 5: Aplicación Web Simple con Flask

In [ ]:
# Ejercicio 5.1: Crear una aplicación web básica
# Nota: Este código está preparado para ejecutarse en un entorno local

app_code = '''
from flask import Flask, jsonify, request, render_template_string
import json
from datetime import datetime

app = Flask(__name__)

# Datos en memoria (en producción usaríamos una base de datos)
datos_app = {
    'posts': [],
    'usuarios': [],
    'visitas': 0
}

# Template HTML simple
template_html = """
<!DOCTYPE html>
<html>
<head>
    <title>API Web Dashboard</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 40px; }
        .container { max-width: 800px; margin: 0 auto; }
        .stat-box { background: #f0f0f0; padding: 20px; margin: 10px 0; border-radius: 5px; }
        .btn { background: #007bff; color: white; padding: 10px 20px; 
               border: none; border-radius: 5px; cursor: pointer; }
        .btn:hover { background: #0056b3; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Dashboard de API Web</h1>
        
        <div class="stat-box">
            <h3>Estadísticas</h3>
            <p><strong>Posts:</strong> {{ stats.posts }}</p>
            <p><strong>Usuarios:</strong> {{ stats.usuarios }}</p>
            <p><strong>Visitas:</strong> {{ stats.visitas }}</p>
            <p><strong>Última actualización:</strong> {{ stats.timestamp }}</p>
        </div>
        
        <div class="stat-box">
            <h3>Acciones</h3>
            <button class="btn" onclick="cargarDatos()">Cargar Datos de API</button>
            <button class="btn" onclick="verPosts()">Ver Posts</button>
        </div>
        
        <div id="contenido"></div>
    </div>
    
    <script>
        function cargarDatos() {
            fetch('/api/cargar-datos', {method: 'POST'})
                .then(response => response.json())
                .then(data => {
                    alert('Datos cargados: ' + data.mensaje);
                    location.reload();
                });
        }
        
        function verPosts() {
            fetch('/api/posts')
                .then(response => response.json())
                .then(data => {
                    let html = '<h3>Posts Recientes</h3>';
                    data.forEach(post => {
                        html += `<div class="stat-box">
                                   <h4>${post.title}</h4>
                                   <p>${post.body.substring(0, 100)}...</p>
                                 </div>`;
                    });
                    document.getElementById('contenido').innerHTML = html;
                });
        }
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    """Página principal del dashboard"""
    datos_app['visitas'] += 1
    
    stats = {
        'posts': len(datos_app['posts']),
        'usuarios': len(datos_app['usuarios']),
        'visitas': datos_app['visitas'],
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    return render_template_string(template_html, stats=stats)

@app.route('/api/posts')
def obtener_posts():
    """API endpoint para obtener posts"""
    return jsonify(datos_app['posts'][:5])  # Primeros 5 posts

@app.route('/api/cargar-datos', methods=['POST'])
def cargar_datos():
    """Carga datos desde la API externa"""
    try:
        # Simular carga de datos
        cliente = ClienteAPI()
        posts = cliente.obtener_posts(10)
        usuarios = cliente.obtener_usuarios_externos(5)
        
        if posts:
            datos_app['posts'] = posts
        if usuarios:
            datos_app['usuarios'] = usuarios
        
        return jsonify({
            'exito': True,
            'mensaje': f'Cargados {len(posts)} posts y {len(usuarios)} usuarios'
        })
    except Exception as e:
        return jsonify({'exito': False, 'mensaje': str(e)})

if __name__ == '__main__':
    app.run(debug=True, port=5000)
'''

# Guardar el código de la aplicación Flask
with open('app_web.py', 'w', encoding='utf-8') as f:
    f.write(app_code)

print("\n=== Aplicación Web Creada ===")
print("Archivo guardado: app_web.py")
print("")
print("Para ejecutar la aplicación:")
print("1. Instala Flask: pip install flask")
print("2. Ejecuta: python app_web.py")
print("3. Abre: http://localhost:5000")
print("")
print("Funcionalidades de la app:")
print("- Dashboard con estadísticas")
print("- Carga de datos desde API externa")
print("- Visualización de posts")
print("- Contador de visitas")

## Parte 6: Ejercicios Prácticos

### Ejercicio 6.1: API Cliente Personalizada
Crea una clase que:
1. Se conecte a una API de clima (OpenWeatherMap)
2. Implemente cache local para evitar llamadas repetidas
3. Maneje errores de conexión graciosamente
4. Guarde los datos en diferentes formatos

### Ejercicio 6.2: Web Scraping Ético
Implementa un scraper que:
1. Extraiga noticias de un sitio web público
2. Respete el robots.txt
3. Implemente delays entre requests
4. Almacene los datos en una base de datos SQLite

### Ejercicio 6.3: API REST Completa
Extiende la aplicación Flask para:
1. Implementar CRUD completo (Create, Read, Update, Delete)
2. Agregar autenticación básica
3. Implementar paginación
4. Agregar validación de datos
5. Crear documentación de API

In [ ]:
# Espacio para soluciones de los ejercicios

# Ejercicio 6.1: API Cliente Personalizada
class ClienteClima:
    def __init__(self, api_key):
        # TODO: Implementar cliente de clima
        pass
    
    def obtener_clima(self, ciudad):
        # TODO: Implementar obtención de clima
        pass

# Ejercicio 6.2: Web Scraping Ético
class ScraperNoticias:
    def __init__(self, base_url):
        # TODO: Implementar scraper
        pass
    
    def extraer_noticias(self):
        # TODO: Implementar extracción
        pass

# Ejercicio 6.3: API REST Completa
# TODO: Extender la aplicación Flask con funcionalidades adicionales

print("¡Completa los ejercicios de desarrollo web!")
print("")
print("Recursos adicionales:")
print("- Documentación de Flask: https://flask.palletsprojects.com/")
print("- Requests: https://requests.readthedocs.io/")
print("- BeautifulSoup: https://beautiful-soup-4.readthedocs.io/")
print("- JSONPlaceholder (API de prueba): https://jsonplaceholder.typicode.com/")